In [ ]:
#!/usr/bin/python

import sys
import pickle
import pandas as pd
import numpy
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
import time
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import AdaBoostClassifier

sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

In [2]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
# features_list is a list of my selected features
# finance_features has finance features I'm interested in
# email_features is a list of email features
features_list = ['poi', 'bonus', 'expenses', 'bon_plus_expenses', 'bon_sal_ratio', \
                'to_msg_ratio', 'from_msg_ratio']
finance_features = ['poi', 'salary', 'bonus', 'long_term_incentive', \
                 'deferred_income', 'expenses', 'total_payments', \
                 'exercised_stock_options', 'restricted_stock', 'other'] 
email_features = ['poi', 'to_messages', 'email_address', 
                 'from_poi_to_this_person', 'from_messages', \
                 'from_this_person_to_poi', 'shared_receipt_with_poi', \
                 'to_msg_ratio', 'from_msg_ratio']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

df = pd.DataFrame.from_records(list(data_dict.values()))
employees = pd.Series(list(data_dict.keys()))

# set the index of df to be the employees series:
df.set_index(employees, inplace=True)

In [3]:
# Find how many missing values are in my selected features
for feature in features_list:
    try:
        print "Number of missing values in " + str(feature) + ": " + str(df[feature].value_counts(dropna=False)[0])
    except:
        print "Created feature: ", str(feature)

Number of missing values in poi: 128
Number of missing values in bonus: 64
Number of missing values in expenses: 51
Created feature:  bon_plus_expenses
Created feature:  bon_sal_ratio
Created feature:  to_msg_ratio
Created feature:  from_msg_ratio


In [4]:
# Create another working dataframe to make new features 

df_new = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).copy()

In [5]:
# from_msg_ratio is ratio messages received from poi to total messages received
df_new['to_msg_ratio'] = df_new.from_this_person_to_poi.divide(df_new.to_messages, axis = 'index')

# create to_msg_ratio by dividing from_this_person_to_poi from to_messages
df_new['from_msg_ratio'] = df_new.from_poi_to_this_person.divide(df_new.from_messages, axis = 'index')

# create a new feature by adding expenses and bonus together
df_new['bon_plus_expenses'] = df_new['bonus'].add(df_new['expenses'], axis = 'index')
# new feature of bonus to salary ratio

df_new['bon_sal_ratio'] = df_new['bonus'].divide(df_new['salary'], axis = 'index')
# new feature of bonus to expenses ratio

In [6]:
# Fill NaN with 0 where operations created NaN in some rows
df_new.fillna(0, inplace = True)

In [7]:
#df_new['bon_plus_expenses']

In [8]:
# create a dictionary from the dataframe
df_dict = df_new.to_dict('index')

In [9]:
# after you create features, the column names will be your new features
# create a list of column names:
new_features_list = df_new.columns.values
new_features_list[:5]

array(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'email_address'], dtype=object)

In [10]:
### Task 2: Remove outliers

# I want to try out classifiers first before removing outliers to see 
# if it would perform any better.

In [11]:
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = df_dict

In [12]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [13]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.

In [14]:
# Trying PCA to combine some features
pca = PCA(n_components = 2)
pca.fit(features)

#print pca.explained_variance_ratio_
print pca.components_
#print pca.explained_variance_
features_pca = pca.transform(features)

[[  6.87843393e-01   3.63301401e-02   7.24949369e-01   4.31046157e-08
   -2.89403891e-11  -2.96706207e-09]
 [  7.19200968e-01  -1.69077155e-01  -6.73916081e-01   5.64282610e-06
    1.15852540e-09  -2.16616278e-07]]


In [15]:
# First one tried is RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 25, min_samples_split = 3, 
                            random_state = 2)
rfc = rfc.fit(features, labels)

print rfc.feature_importances_

[ 0.15654285  0.23536453  0.13178955  0.13761229  0.15467439  0.18401639]


In [18]:
# Next, Adaboost

#abc = AdaBoostClassifier()
#abc = abc.fit(features, labels)

#print abc.feature_importances_

# After reading a forum post, it looked like decision tree classifiers 
# work better in this setting

In [16]:
# Also trying a decision tree classifier because tree classifiers make sense here

dc = DecisionTreeClassifier()
dc = dc.fit(features, labels)

print dc.feature_importances_

[ 0.05627706  0.40415264  0.42795414  0.          0.          0.11161616]


In [28]:
# using selectkbest to do feature selection

selection = SelectKBest(chi2, k = 2)

In [43]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info:
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
# look up parameters to use for cross validation here 
# random_state is to bring consistency to results
# results to best_params_ was variable before adding that parameter
# If you uncomment to run these lines of code, it may take a while
# Added start and end times to see how long this all takes because
# this exhaustive method has been taking forever

start_gridcv_rfc = time.time()

rbc_param_grid = {'n_estimators': [1,2, 3, 10, 100], 
                 'min_samples_split': [2, 3, 5],
                 'random_state': [2],
                 'max_features': [1, 2, 3],
                 'max_depth' : [2, 3, 5, 10, 50],
                 'min_samples_leaf': [1, 2, 3, 10]}

grid_cv_rfc = GridSearchCV(estimator = rfc, param_grid = rbc_param_grid, cv = 5,
                          n_jobs = 3)
grid_cv_rfc.fit(features, labels)

grid_cv_rfc.best_params_

end_gridcv_rfc = time.time()
print "Minutes elapse: " + str((float(end_gridcv_rfc - start_gridcv_rfc) / 60))

In [ ]:
# gridsearchcv for decisiontreeclassifier

start_gridcv_dc = time.time()

dc_param_grid = {'min_samples_split' : [2, 3, 4, 5, 10, 50],
                 'max_features' : [1, 2, 3, 4, 'auto', 'sqrt', 'log2'],
                 'min_samples_leaf': [1, 2, 3, 10, 20],
                'random_state' : [2]
                }

grid_cv_dc = GridSearchCV(estimator = dc, param_grid = dc_param_grid)
grid_cv_dc.fit(features, labels)

grid_cv_dc.best_params_

end_gridcv_dc = time.time()

print "Minutes elapse: " + str((float(end_gridcv_dc - start_gridcv_dc) / 60))

In [22]:
# kmeans cross validation

kf_rbc = KFold(n_splits = 10, shuffle=True, random_state = 2)
kf_rbc.split(features, labels)
print kf_rbc

KFold(n_splits=10, random_state=2, shuffle=True)


In [23]:
#for train_indicies, test_indicies in kf_rbc:
#    features_train = [data[ii] for ii in train_indicies]

In [33]:
# check accuracy, precision, and recall of my classifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

#print accuracy_score(features_test, labels_test)

In [44]:
print classification_report(labels_train, grid_cv_dc.best_estimator_.predict(features_train))

             precision    recall  f1-score   support

        0.0       0.86      1.00      0.93        70
        1.0       0.00      0.00      0.00        11

avg / total       0.75      0.86      0.80        81



In [45]:
print classification_report(labels_test, grid_cv_dc.best_estimator_.predict(features_test))

             precision    recall  f1-score   support

        0.0       0.81      1.00      0.89        29
        1.0       0.00      0.00      0.00         7

avg / total       0.65      0.81      0.72        36



In [46]:
print classification_report(labels_train, grid_cv_rfc.best_estimator_.predict(features_train))

             precision    recall  f1-score   support

        0.0       0.92      1.00      0.96        70
        1.0       1.00      0.45      0.62        11

avg / total       0.93      0.93      0.91        81



In [47]:
print classification_report(labels_test, grid_cv_rfc.best_estimator_.predict(features_test))

             precision    recall  f1-score   support

        0.0       0.85      1.00      0.92        29
        1.0       1.00      0.29      0.44         7

avg / total       0.88      0.86      0.83        36



In [31]:
# Assign clf to classifer chosen after testing

clf = RandomForestClassifier(min_samples_split = 2, n_estimators = 100,
                            random_state = 2, max_depth = 5, min_samples_leaf = 3,
                            max_features = 1)
#clf.fit(features, labels)

clf = make_pipeline(selection, clf)
clf.fit(features_train, labels_train)

Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=2, score_func=<function chi2 at 0x113ae45f0>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurit...stimators=100, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False))])

In [26]:
clf = DecisionTreeClassifier(min_samples_split = 2, random_state = 2,
                            max_features = 3, min_samples_leaf = 1)

clf.fit(features_train, labels_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=2, splitter='best')

In [32]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)
